In [16]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from prophet import Prophet
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.io as pio
pio.templates.default = "plotly_white"


In [ ]:

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Load dataset
file_path = "/content/drive/MyDrive/Covid-19 Analysis - Intellipaat/covid_19_clean_complete.csv"
df = pd.read_csv(file_path)


In [ ]:
# Convert Date column to datetime format
df["Date"] = pd.to_datetime(df["Date"])


In [ ]:
# Fill missing values
df.fillna(0, inplace=True)

In [ ]:
# Aggregate Global Data
global_data = df.groupby("Date")[["Confirmed", "Deaths", "Recovered"]].sum().reset_index()

In [ ]:
# Compute Active Cases
global_data["Active"] = global_data["Confirmed"] - (global_data["Deaths"] + global_data["Recovered"])

In [ ]:
# Growth Rate Calculation
global_data["New Cases"] = global_data["Confirmed"].diff()
global_data["New Deaths"] = global_data["Deaths"].diff()
global_data["New Recovered"] = global_data["Recovered"].diff()

In [ ]:
# Visualization: COVID-19 Trends
fig1 = px.line(global_data, x="Date", y=["Confirmed", "Deaths", "Recovered", "Active"],
               title="Global COVID-19 Trends", labels={"value": "Cases", "variable": "Category"})
fig1.show()

In [ ]:
# Top 10 Affected Countries
latest_date = df["Date"].max()
top_countries = df[df["Date"] == latest_date].groupby("Country/Region")["Confirmed"].sum().nlargest(10)

fig2 = px.bar(top_countries, x=top_countries.index, y=top_countries.values,
              title="Top 10 Countries with Highest Cases", labels={"x": "Country", "y": "Confirmed Cases"})
fig2.show()

In [ ]:
# Prepare Data for Forecasting
forecast_data = global_data[["Date", "Confirmed"]].rename(columns={"Date": "ds", "Confirmed": "y"})


In [ ]:
# Train Prophet Model
model = Prophet()
model.fit(forecast_data)

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpmw2jl1d5/x0dts977.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpmw2jl1d5/_ylqz1a5.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=60904', 'data', 'file=/tmp/tmpmw2jl1d5/x0dts977.json', 'init=/tmp/tmpmw2jl1d5/_ylqz1a5.json', 'output', 'file=/tmp/tmpmw2jl1d5/prophet_model5jvmbirp/prophet_model-20250211092732.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
09:27:32 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
09:27:32 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


In [ ]:
# Predict Next 30 Days
future = model.make_future_dataframe(periods=30)
forecast = model.predict(future)

In [ ]:
# Visualization: Forecasting Future Cases
fig3 = px.line(forecast, x="ds", y="yhat", title="COVID-19 Forecast (Next 30 Days)",
               labels={"yhat": "Predicted Cases", "ds": "Date"})
fig3.show()

In [ ]:
# Country-wise Time Series Visualization
country = "India"  # Change to any country
df_country = df[df["Country/Region"] == country].groupby("Date")["Confirmed"].sum().reset_index()

fig4 = px.line(df_country, x="Date", y="Confirmed",
               title=f"COVID-19 Trend in {country}", labels={"Confirmed": "Cases"})
fig4.show()

print("Detailed COVID-19 Analysis Completed Successfully!")


Detailed COVID-19 Analysis Completed Successfully!
